In [1]:
import glob
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA
import numpy as np

2023-11-29 01:33:03.063652: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 01:33:04.489707: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 01:33:08.246857: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 01:33:08.507548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 01:33:41.746216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
def load_shape_info(dir):
    import pickle
    with open(dir + '/pickleshapes', 'rb') as file:
        # Load the data from the file
        shapes = pickle.load(file)
        return shapes
def _lab_labels(all_labels):
        #, this is an embedded function called from below
        labels = {}
        labels['comp_labels'] = tf.one_hot(tf.cast(all_labels[0],tf.int32),3)
        labels['amp_labels'] = tf.one_hot(tf.cast(all_labels[1],tf.int32),10)
        labels['torque_labels'] = tf.one_hot(tf.cast(all_labels[2],tf.int32),5)
        labels['joint_labels'] = tf.one_hot(tf.cast(all_labels[3],tf.int32),3)
        return labels        

In [3]:
def load_tf_data(folder,shape_info):
    tf_records = glob.glob(os.path.join(folder, '*.tf'))
    dataset = tf.data.TFRecordDataset(tf_records)
    def _parse_function(example_proto):
        feature_description = {
            'ts': tf.io.FixedLenFeature(shape_info[0], tf.float32),
            'labels': tf.io.FixedLenFeature(shape_info[1], tf.float32),
            'pos_labels': tf.io.FixedLenFeature(shape_info[2], tf.float32)
        }
        #example = tf.io.parse_single_example(example_proto, feature_description)
        example = tf.io.parse_example(example_proto, feature_description)
        all_labels = tf.cast(example['labels'], tf.float32)
        #all_labels = example['labels']
        labels = _lab_labels(all_labels)
        if 'pos_labels' in example.keys():
            pos_labels = tf.cast(example['pos_labels'], tf.float32)
            labels['pos_labels'] = pos_labels
        ts = example['ts']
        #labels = all_labels
        return ts, labels['comp_labels']
        #return example['ts'], labels
    dataset = dataset.map(_parse_function)
    dataset = dataset.batch(batch_size=500, drop_remainder = False)
    return dataset

In [4]:
#dir_base = '/scratch/user/swati/Capstone/'
#dir_data = 'RoughCut_Datasets' /
dir_base = '/scratch/group/statconsult/Output_hema/'
dir_data = 'Data6_Copy/'
dir = dir_base + dir_data + '/'
train_shape_info = load_shape_info(dir + 'train')
test_shape_info = load_shape_info(dir + 'predict')
predict_shape_info = load_shape_info(dir + 'validate')
train_data = load_tf_data(dir + 'train',train_shape_info)
test_data = load_tf_data(dir + 'predict',test_shape_info)
predict_data = load_tf_data(dir + 'validate',predict_shape_info)

In [7]:
import tensorflow as tf
from tensorflow import keras

# # Define the model
# model = keras.Sequential()

# # Flatten the input data
# model.add(keras.layers.Flatten(input_shape=(1000, 50)))

# # Add a couple of dense layers
# model.add(keras.layers.Dense(128, activation='relu'))
# model.add(keras.layers.Dense(64, activation='relu'))

# # Output layer with 4 units (assuming you have 4 classes for classification)
# model.add(keras.layers.Dense(3, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Print the model summary
# model.summary()


# model = keras.Sequential()
# model.add(keras.layers.Flatten(input_shape=(1000, 50)))
# model.add(keras.layers.Dense(128, activation='relu'))
# model.add(keras.layers.Dropout(0.5))  # Adding dropout for regularization
# model.add(keras.layers.Dense(64, activation='relu'))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Dense(3, activation='softmax'))

# # Compile the model with a lower learning rate
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#model.summary()
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(1000, 105)))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))  # Adding dropout for regularization
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# model = keras.Sequential([
#     layers.Flatten(input_shape=(1000, 50)),
#     layers.BatchNormalization(),
    
#     layers.Dense(256, activation='relu'),
#     layers.Dropout(0.5),

#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.5),

#     layers.Dense(64, activation='relu'),
#     layers.Dropout(0.5),

#     layers.Dense(3, activation='softmax')
# ])

# # Use a different optimizer (SGD) and add learning rate scheduling
# opt = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
# model.compile(optimizer=opt,
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 105000)            0         
                                                                 
 dense_3 (Dense)             (None, 128)               13440128  
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                                 
Total params: 13448579 (51.30 MB)
Trainable params: 13

In [8]:
# Set the number of epochs
num_epochs = 100

# Train the model
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-5)

#history = model1.fit(train_reduced_dataset, callbacks=[lr_scheduler],epochs=num_epochs, validation_data=val_reduced_dataset)
history = model.fit(train_data,epochs=num_epochs, validation_data=predict_data)

Epoch 1/100
216/216 [==============================] - 36s 158ms/step - loss: 1.1472 - accuracy: 0.3369 - val_loss: 1.0135 - val_accuracy: 0.4167
Epoch 2/100
216/216 [==============================] - 27s 124ms/step - loss: 0.9083 - accuracy: 0.5474 - val_loss: 0.8821 - val_accuracy: 0.5273
Epoch 3/100
216/216 [==============================] - 27s 124ms/step - loss: 0.4749 - accuracy: 0.8276 - val_loss: 0.9421 - val_accuracy: 0.7595
Epoch 4/100
216/216 [==============================] - 27s 124ms/step - loss: 0.1912 - accuracy: 0.9486 - val_loss: 1.1538 - val_accuracy: 0.8233
Epoch 5/100
216/216 [==============================] - 24s 112ms/step - loss: 0.1027 - accuracy: 0.9740 - val_loss: 1.2867 - val_accuracy: 0.8527
Epoch 6/100
216/216 [==============================] - 25s 116ms/step - loss: 0.0666 - accuracy: 0.9834 - val_loss: 1.4073 - val_accuracy: 0.8603
Epoch 7/100
216/216 [==============================] - 25s 118ms/step - loss: 0.0496 - accuracy: 0.9867 - val_loss: 1.5214 -

In [9]:
# Evaluate the model on your test data using the dataset with 'comp_labels' only
test_loss, test_accuracy = model.evaluate(test_data)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

12/12 [==============================] - 6s 472ms/step - loss: 0.2418 - accuracy: 0.9422
Test Accuracy: 94.22%


In [60]:
pip install sklearn

  Using cached sklearn-0.0.post11.tar.gz (3.6 kB)
    ERROR: Command errored out with exit status 1:
     command: /scratch/user/swati/pip_envs/my_notebook-Python/3.8.2/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/job.9351895/pip-install-7799e_kd/sklearn/setup.py'"'"'; __file__='"'"'/tmp/job.9351895/pip-install-7799e_kd/sklearn/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/job.9351895/pip-install-7799e_kd/sklearn/pip-egg-info
         cwd: /tmp/job.9351895/pip-install-7799e_kd/sklearn/
    Complete output (18 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
    rather than 'sklearn' for pip commands.
    
    Here is how to fix this error in the main use cases:
    - use 'pip install scikit-learn' rather than 'pip install sklearn'
    - replace 'sklearn' by 'scikit-learn' in your pip

In [61]:
pip install scikit-learn

You should consider upgrading via the '/scratch/user/swati/pip_envs/my_notebook-Python/3.8.2/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [80]:
!pip list | grep 'tensorflow\|numpy'

numpy                         1.24.3    
tensorflow                    2.13.1    
tensorflow-estimator          2.13.0    
tensorflow-io-gcs-filesystem  0.34.0    
You should consider upgrading via the '/scratch/user/swati/pip_envs/my_notebook-Python/3.8.2/bin/python -m pip install --upgrade pip' command.
